In [8]:
import pandas as pd
import os
from PIL import Image

# Step 1: Read a CSV file into a DataFrame
csv_path = '/content/drive/MyDrive/License Plates/CSV/Licplatesdetection_train.csv'  # Replace with your CSV file path
df = pd.read_csv(csv_path)

# Step 2: Get image dimensions from a folder
image_folder = '/content/drive/MyDrive/Home2/datasets/License-Plate-Detector-2/train/images'  # Replace with your image folder path

# Add new columns for image dimensions, filename, and class
df['filename'] = None
df['class'] = 'license_plate'  # Default class; replace if needed
df['img_width'] = None
df['img_height'] = None

# Iterate through images to get dimensions and update DataFrame
for i, image_name in enumerate(sorted(os.listdir(image_folder))):
    image_path = os.path.join(image_folder, image_name)
    if not os.path.isfile(image_path):
        continue

    # Get image dimensions
    with Image.open(image_path) as img:
        img_width, img_height = img.size

    # Update the DataFrame
    if i < len(df):  # Ensure we don't exceed the DataFrame rows
        df.loc[i, 'filename'] = image_name
        df.loc[i, 'img_width'] = img_width
        df.loc[i, 'img_height'] = img_height

# Step 3: Prepare YOLO-normalized values
df['x_center'] = (df['xmin'] + df['xmax']) / 2 / df['img_width']
df['y_center'] = (df['ymin'] + df['ymax']) / 2 / df['img_height']
df['width'] = (df['xmax'] - df['xmin']) / df['img_width']
df['height'] = (df['ymax'] - df['ymin']) / df['img_height']

# Step 4: Save the final CSV file
output_csv_path = '/content/drive/MyDrive/License Plates/CSV/normalized_annotations.csv'
df[['filename', 'class', 'x_center', 'y_center', 'width', 'height']].to_csv(output_csv_path, index=False)

# Step 5: Create YOLO annotation files
output_txt_folder = '/content/drive/MyDrive/License Plates/yolo_annotations_detection'
os.makedirs(output_txt_folder, exist_ok=True)

# Create a .txt file for each image
for _, row in df.iterrows():
    if pd.isna(row['filename']):  # Skip rows without filenames
        continue
    txt_file_path = os.path.join(output_txt_folder, f"{os.path.splitext(row['filename'])[0]}.txt")
    with open(txt_file_path, 'w') as f:
        f.write(f"0 {row['x_center']} {row['y_center']} {row['width']} {row['height']}\n")


In [14]:
# Step 4: Save the final CSV file
output_csv_path = '/content/drive/MyDrive/License Plates/CSV/normalized_annotations.csv'
df.to_csv(output_csv_path, index=False)

In [9]:
df.head()

,img_id,ymin,xmin,ymax,xmax,filename,class,img_width,img_height,x_center,y_center,width,height
0,1.jpg,276,94,326,169,1.jpg,license_plate,850,477,0.154706,0.631027,0.088235,0.104822
1,10.jpg,311,395,344,444,10.jpg,license_plate,639,479,0.656495,0.683716,0.076682,0.068894
2,100.jpg,406,263,450,434,100.jpg,license_plate,729,547,0.478052,0.78245,0.234568,0.080439
3,101.jpg,283,363,315,494,101.jpg,license_plate,850,477,0.504118,0.626834,0.154118,0.067086
4,102.jpg,139,42,280,222,102.jpg,license_plate,409,547,0.322738,0.382998,0.440098,0.25777


In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame from the previous code

# Check for negative values in specific columns
negative_values_present = False
columns_to_check = ['xmin', 'ymin', 'xmax', 'ymax', 'img_width', 'img_height', 'x_center', 'y_center', 'width', 'height']  # Add other columns as needed

for col in columns_to_check:
    if (df[col] < 0).any():
        negative_values_present = True
        print(f"Negative values found in column: {col}")
        # Optionally, print the rows with negative values
        print(df[df[col] < 0])


if not negative_values_present:
    print("No negative values found in specified columns.")

No negative values found in specified columns.


In [ ]:
# Define a threshold for border proximity (e.g., 5 pixels)
border_threshold = 5

# Filter rows where xmin, ymin, xmax, or ymax are within the border threshold
border_rows = df[
    (df['xmin'] < border_threshold) |
    (df['ymin'] < border_threshold) |
    (df['xmax'] > df['img_width'] - border_threshold) |
    (df['ymax'] > df['img_height'] - border_threshold)
]

border_rows

,img_id,ymin,xmin,ymax,xmax,filename,class,img_width,img_height,x_center,y_center,width,height
6,104.jpg,482,511,547,623,104.jpg,license_plate,729,547,0.777778,0.940585,0.153635,0.11883
222,3.jpg,525,216,547,397,3.jpg,license_plate,729,547,0.420439,0.97989,0.248285,0.040219
245,32.jpg,324,292,437,409,32.jpg,license_plate,410,547,0.854878,0.695612,0.285366,0.206581
283,354.jpg,310,240,359,386,354.jpg,license_plate,479,359,0.653445,0.931755,0.304802,0.13649
427,484.jpg,502,300,547,463,484.jpg,license_plate,729,547,0.52332,0.958867,0.223594,0.082267
744,77.jpg,336,1,419,116,77.jpg,license_plate,728,547,0.080357,0.690128,0.157967,0.151737
790,810.jpg,391,651,456,730,810.jpg,license_plate,733,547,0.942019,0.774223,0.107776,0.11883
835,851.jpg,409,265,546,499,851.jpg,license_plate,729,547,0.524005,0.872943,0.320988,0.250457


In [ ]:
# Define a threshold for border proximity (e.g., 5 pixels)
# This is now based on x_center, y_center, width and height
border_threshold = 0.01 # Adjust as needed

# Calculate the boundaries for x and y center considering width and height
df['x_min_bound'] = df['x_center'] - (df['width'] / 2)
df['x_max_bound'] = df['x_center'] + (df['width'] / 2)
df['y_min_bound'] = df['y_center'] - (df['height'] / 2)
df['y_max_bound'] = df['y_center'] + (df['height'] / 2)


# Filter rows based on the new boundaries
border_rows = df[
    (df['x_min_bound'] < border_threshold) |
    (df['x_max_bound'] > 1 - border_threshold) |
    (df['y_min_bound'] < border_threshold) |
    (df['y_max_bound'] > 1 - border_threshold)
]

border_rows

,img_id,ymin,xmin,ymax,xmax,filename,class,img_width,img_height,x_center,y_center,width,height,x_min_bound,x_max_bound,y_min_bound,y_max_bound
6,104.jpg,482,511,547,623,104.jpg,license_plate,729,547,0.777778,0.940585,0.153635,0.11883,0.70096,0.854595,0.88117,1.0
222,3.jpg,525,216,547,397,3.jpg,license_plate,729,547,0.420439,0.97989,0.248285,0.040219,0.296296,0.544582,0.959781,1.0
245,32.jpg,324,292,437,409,32.jpg,license_plate,410,547,0.854878,0.695612,0.285366,0.206581,0.712195,0.997561,0.592322,0.798903
283,354.jpg,310,240,359,386,354.jpg,license_plate,479,359,0.653445,0.931755,0.304802,0.13649,0.501044,0.805846,0.86351,1.0
427,484.jpg,502,300,547,463,484.jpg,license_plate,729,547,0.52332,0.958867,0.223594,0.082267,0.411523,0.635117,0.917733,1.0
744,77.jpg,336,1,419,116,77.jpg,license_plate,728,547,0.080357,0.690128,0.157967,0.151737,0.001374,0.159341,0.61426,0.765996
790,810.jpg,391,651,456,730,810.jpg,license_plate,733,547,0.942019,0.774223,0.107776,0.11883,0.888131,0.995907,0.714808,0.833638
835,851.jpg,409,265,546,499,851.jpg,license_plate,729,547,0.524005,0.872943,0.320988,0.250457,0.363512,0.684499,0.747715,0.998172
